In [1]:
import praw
import pandas as pd
from datetime import datetime
from collections import defaultdict
from datetime import timedelta
import itertools
import ipywidgets as widgets
import re
from IPython.display import HTML

reddit = praw.Reddit(
    user_agent="reports-tool",
    site_name="ssc"
)

pd.set_option('display.max_colwidth', 150)

In [21]:
queue = list(reddit.subreddit('mod').mod.reports(limit=None))

In [31]:
def get_reports(comment):
    mod_reports = [(reason, 1) for (reason, _) in comment.mod_reports]
    
    reports_dict = defaultdict(lambda: 0)
    for reports in comment.user_reports + mod_reports:
        reports_dict[reports[0]] += reports[1]
    return reports_dict

problematic_comments = []

def comment_to_dict(comment):
    try:
        reports = get_reports(comment)
        quality_reports = sum(count for reason, count in reports.items() if "quality" in reason)
        non_quality_reports = sum(count for reason, count in reports.items() if "quality" not in reason)
        return {
            "author": comment.author.name,
            "score": comment.score,
            "reports": reports,
            "quality": quality_reports,
            "non-quality": non_quality_reports,
            "age": datetime.now() - datetime.fromtimestamp(comment.created_utc),
            "body": comment.body,
            "object": comment,
            "permalink": "https://www.reddit.com" + comment.permalink
        }
    except Exception as e:
        print("Whoops! {0} {1}".format(comment, e))
        problematic_comments.append(comment)
        return {}

comments = filter(lambda x: isinstance(x, praw.models.Comment) and x.author is not None, queue)
df = pd.DataFrame.from_dict([comment_to_dict(c) for c in comments]).dropna()
df_filtered = df

In [32]:
df_filtered.groupby('author').sum().sort_values(['non-quality', 'score'], ascending=[False, True])[['non-quality', 'quality', 'score']]

,non-quality,quality,score
author,,,
qualia_of_mercy,10,0,55
Impassionata,7,0,-11
queensnyatty,6,0,-9
GavinSkulldrinker,6,0,16
SkoomaDentist,5,0,-4
SillyEnthusiasm,4,0,0
895158,4,0,4
working_class_shill,3,0,3
Jiro_T,3,0,8


In [33]:
df.groupby('author').sum().sort_values(['quality', 'non-quality'], ascending=[False, True])[['quality', 'non-quality']]

,quality,non-quality
author,,
TrannyPornO,4,1
orangejake,1,0
baj2235,1,3
Glopknar,0,1
MomentarySanityLapse,0,1
Nwallins,0,1
PmMeExistentialDread,0,1
brberg,0,1
chipsa,0,1


In [34]:
current_reports = df[(df['age'] <= timedelta(days=3.0))].sort_values(['non-quality', 'score'], ascending=[False, True])
with_hyperlink = current_reports.assign(axis=1, permalink=current_reports['permalink'].map(lambda link: '<a href="{0}?context=1">link</a>'.format(link)))
HTML(with_hyperlink[['non-quality', 'score', 'author', 'age', 'body', 'permalink']].to_html(escape=False))

,non-quality,score,author,age,body,permalink
14,7,-11,Impassionata,0 days 21:02:29.827026,">Two years ago, I noticed that, unlike me, a lot of otherwise-respectable adults I know don't seem to have a ""you win some, you lose some"" in resp...",link
26,5,3,GavinSkulldrinker,1 days 07:26:58.827026,"Well, that shuts me up re: prison, if accurate. But since apparently the entire rest of the middle East and North Africa can apparently just walt...",link
33,3,-10,SkoomaDentist,1 days 09:14:22.828031,Hence my quip a long time ago that they should just give everyone a fair advance warning and nuke the whole place unlivable radioactive wasteland....,link
32,3,3,working_class_shill,1 days 08:31:12.828031,You don't need evidence when strawmanning leftists,link
23,3,17,qualia_of_mercy,1 days 06:19:57.827026,"Ah, so you are aware of Egypt's existence. Is there really a treaty saying Egypt is not allowed to let Gazans cross its own border? That would be ...",link
27,3,19,qualia_of_mercy,1 days 07:29:19.827026,"Pakistan. Or every nation in the former Yugoslavia. There, that was easy.\n\nAs for that dig about ""chosen people,"" can I just say that from anti-...",link
45,3,19,baj2235,2 days 09:08:06.828031,>The quality of pornography that most distorts men's minds is that the women in it are friendly.\n\nWat? No.\n\n 1) Its like you are trying to rus...,link
9,2,-5,queensnyatty,0 days 12:47:45.827026,I imagine it’ll be my go-to for a long time when I need an example of a perfectly worthless comment.,link
1,2,-1,Ilforte,0 days 01:29:23.827026,"It is no different of course, that’s the whole point, ain’t nobody getting tricked by me. I am, in effect, calling him a retard, in part to illust...",link
28,2,-1,SillyEnthusiasm,1 days 07:30:17.827026,Please just come out and advocate for apartheid.,link


In [35]:
# Finding patterns
df[df['non-quality'] >= 2.0 ].sort_values(['author', 'non-quality'], ascending=[True, False])

,age,author,body,non-quality,object,permalink,quality,reports,score
26,1 days 07:26:58.827026,GavinSkulldrinker,"Well, that shuts me up re: prison, if accurate. But since apparently the entire rest of the middle East and North Africa can apparently just walt...",5,dz0vuy3,https://www.reddit.com/r/slatestarcodex/comments/8jc7i6/culture_war_roundup_for_the_week_of_may_14_2018/dz0vuy3/,0,"{'Low-effort comment': 2, '“white-guilt the Swedes” is inflammatory language that wouldn’t be tolerated if it was from the left': 1, 'Waging cultu...",3
1,0 days 01:29:23.827026,Ilforte,"It is no different of course, that’s the whole point, ain’t nobody getting tricked by me. I am, in effect, calling him a retard, in part to illust...",2,dz3myrq,https://www.reddit.com/r/slatestarcodex/comments/8jc7i6/culture_war_roundup_for_the_week_of_may_14_2018/dz3myrq/,0,"{'Whiny crap.': 1, 'Antagonizing other users': 1}",-1
14,0 days 21:02:29.827026,Impassionata,">Two years ago, I noticed that, unlike me, a lot of otherwise-respectable adults I know don't seem to have a ""you win some, you lose some"" in resp...",7,dz1zfre,https://www.reddit.com/r/slatestarcodex/comments/8jc7i6/culture_war_roundup_for_the_week_of_may_14_2018/dz1zfre/,0,"{'Low-effort comment': 2, 'Antagonizing other users': 2, 'Waging culture war': 2, 'Be clear about what you mean or shut up': 1}",-11
39,1 days 12:36:37.828031,Jiro_T,">Yesterday the US opened its new embassy in Jerusalem, resulting in massive Palestinian protests and violence in Gaza with 55 killed and 2000 woun...",2,dz0dk7h,https://www.reddit.com/r/slatestarcodex/comments/8jc7i6/culture_war_roundup_for_the_week_of_may_14_2018/dz0dk7h/,0,{'Waging culture war': 2},7
24,1 days 06:49:11.827026,SillyEnthusiasm,If Zionism were anything but racial superiority dressed up in religion you might have a point. If Israel were striving to be a secular state for J...,2,dz0yhmx,https://www.reddit.com/r/slatestarcodex/comments/8jc7i6/culture_war_roundup_for_the_week_of_may_14_2018/dz0yhmx/,0,"{'Antagonizing other users': 1, 'Same thread as Skoomadentist and Gavinskulldrinker. Perhaps a general warnign in order for everyone?': 1}",1
28,1 days 07:30:17.827026,SillyEnthusiasm,Please just come out and advocate for apartheid.,2,dz0vmh7,https://www.reddit.com/r/slatestarcodex/comments/8jc7i6/culture_war_roundup_for_the_week_of_may_14_2018/dz0vmh7/,0,{'Antagonizing other users': 2},-1
33,1 days 09:14:22.828031,SkoomaDentist,Hence my quip a long time ago that they should just give everyone a fair advance warning and nuke the whole place unlivable radioactive wasteland....,3,dz0ohzk,https://www.reddit.com/r/slatestarcodex/comments/8jc7i6/culture_war_roundup_for_the_week_of_may_14_2018/dz0ohzk/,0,"{'Call for violence': 1, 'SkoomaDentist behaving very poorly in this thread. Many reports, this is the worst comment.': 1, 'Fuck that': 1}",-10
30,1 days 07:57:16.828031,SkoomaDentist,"Because ""jewish state"" in that area equals ""apartheid"" in practise.",2,dz0toqb,https://www.reddit.com/r/slatestarcodex/comments/8jc7i6/culture_war_roundup_for_the_week_of_may_14_2018/dz0toqb/,0,"{'Low-effort comment': 1, 'Same thread as SkoomaDentist's report below.': 1}",6
45,2 days 09:08:06.828031,baj2235,>The quality of pornography that most distorts men's minds is that the women in it are friendly.\n\nWat? No.\n\n 1) Its like you are trying to rus...,3,dyyo1g8,https://www.reddit.com/r/slatestarcodex/comments/8jc7i6/culture_war_roundup_for_the_week_of_may_14_2018/dyyo1g8/,0,"{'Less of this sassy mod response please': 1, 'Abusing mod privileges': 1, 'Legbeard': 1}",19
23,1 days 06:19:57.827026,qualia_of_mercy,"Ah, so you are aware of Egypt's existence. Is there really a treaty saying Egypt is not allowed to let Gazans cross its own border? That would be ...",3,dz10hr6,https://www.reddit.com/r/slatestarcodex/comments/8jc7i6/culture_war_roundup_for_the_week_of_may_14_2018/dz10hr6/,0,"{'Antagonizing other users': 2, 'Same thread as Skoomadentist and Gavinskulldrinker. Bad

In [36]:
#df[(df['age'] >= timedelta(days=3.0)) & (df['quality'] == 0)].sort_values(['non-quality', 'age'], ascending=[False, True])
#df[(df['age'] >= timedelta(days=10.0)) & (df['quality'] == 0)].apply(lambda comment: print(comment['object'].mod.approve()), axis=1)

In [39]:
def is_user_authored_text(s):
    return len(s) > 0 and s[0] not in [">"]

def first_non_quote_line(s):
    try:
        return next(x for x in s.splitlines() if is_user_authored_text(x))
    except Exception as e:
        print("Comment is all shit?")
        return s

def first_n_words(s, n):
    return " ".join(s.split(" ")[:n])


import re
def sanitize(s):
    without_links = re.sub(r"\[([^\[]*)\]\(http.*\)", r"\1", s)
    without_opening_brackets = re.sub(r"\[", r"\[", without_links)
    return without_opening_brackets

def make_blurb(comment):
    body_blurb = sanitize(first_n_words(first_non_quote_line(comment['body']), 20))
    return('/u/{0}: ["{1}..."](https://www.reddit.com{2}?context=3&sort=best)'.format(comment['author'], body_blurb, comment['object'].permalink))

filtered = df_filtered[df_filtered['quality'] >= 1]

pearled = filtered.assign(pearl_ratio=filtered['quality']/filtered['score']).sort_values("pearl_ratio", ascending=False)

df_filtered[df_filtered['quality'] >= 1]
blurbed = pearled.assign(blurb=pearled.apply(axis=1, func=make_blurb))
print('\n\n'.join(["{0}".format(x) for x in blurbed['blurb'].tolist()]))

pearled

/u/orangejake: ["I see it as similar to something like /r/exmorman, but for rationalism. I don't personally browse there that often, but..."](https://www.reddit.com/r/slatestarcodex/comments/8jc7i6/culture_war_roundup_for_the_week_of_may_14_2018/dz3bajk/?context=3&sort=best)

/u/baj2235: ["Replying only because this has seen several reports, and I don't want it ending up back in the queue after..."](https://www.reddit.com/r/slatestarcodex/comments/8jc7i6/culture_war_roundup_for_the_week_of_may_14_2018/dz1mupw/?context=3&sort=best)

/u/TrannyPornO: ["  ..."](https://www.reddit.com/r/slatestarcodex/comments/8jc7i6/culture_war_roundup_for_the_week_of_may_14_2018/dyzhy2b/?context=3&sort=best)


,age,author,body,non-quality,object,permalink,quality,reports,score,pearl_ratio
5,0 days 04:33:00.827026,orangejake,"I see it as similar to something like /r/exmorman, but for rationalism. I don't personally browse there that often, but when I have it's been a ni...",0,dz3bajk,https://www.reddit.com/r/slatestarcodex/comments/8jc7i6/culture_war_roundup_for_the_week_of_may_14_2018/dz3bajk/,1,{'Actually a quality contribution': 1},8,0.125
17,1 days 00:28:36.827026,baj2235,"Replying only because this has seen several reports, and I don't want it ending up back in the queue after I've cleared it (Ignoring reports is ch...",0,dz1mupw,https://www.reddit.com/r/slatestarcodex/comments/8jc7i6/culture_war_roundup_for_the_week_of_may_14_2018/dz1mupw/,1,{'Actually a quality contribution': 1},8,0.125
44,2 days 01:29:26.828031,TrannyPornO,>And it's the same thing with men; they're not interested in a guy unless someone else is already interested. \n \nYou have no idea how right yo...,0,dyzhy2b,https://www.reddit.com/r/slatestarcodex/comments/8jc7i6/culture_war_roundup_for_the_week_of_may_14_2018/dyzhy2b/,4,{'Actually a quality contribution': 4},32,0.125


In [ ]:
#blurbed.apply(lambda comment: print(comment['object'].mod.approve()), axis=1)

In [10]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]
        
formatted_comments = ["{0}".format(x) for x in blurbed['blurb'].tolist()]
#for l in chunks(formatted_comments, 8):
#    thread.reply("\n\n".join(l))

NameError: name 'blurbed' is not defined